# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию

In [33]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter
m = Mystem()

In [34]:
data=pd.read_csv('/datasets/data.csv')
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


In [35]:
# общая  информация о таблице:
data.info(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [36]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


In [37]:
#посчитаем сколько уникальных значений с кол-вом детей -1 и 20:
data['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [38]:
#посчитаем кол-во людей с возрастом 0:

data[data['dob_years']==0].count()[0]

101

**Вывод**

Количество значений в столбцах `'days_employed'` и `'total_income'` различается от всех остальных. Значит, в данных есть пропущенные значения.Которые нужно заполнить, либо удалить.


Столбцам `'days_employed' и 'total_income'` стоит присвоить целочисленные значения.

В столбце `'education'` не однотипные значения.Привести к единому стилю.

В столбце `dob_years` минимальный возраст 0 нужно заменить, скорее всего это ошибка так же как и в столбце `children` минимальное кол-во детей -1 и максимальное 20. Вероятнее всего , что указали 20 детей нужно исправить на кол-во 2, а -1 на 0 детей.



## Шаг 2. Предобработка данных

### Обработка пропусков

In [39]:
# подсчёт пропусков
data.isna().sum()  

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [40]:
#проверим кол-во пропусков по типу занятости:
data[data['total_income'].isna()]['income_type'].value_counts()

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

In [41]:
#заполним пропуски в 'total_income' медианным значением с группировкой по типу занятости и образованию:
data['days_employed']=data['days_employed'] \
    .fillna(data.groupby(['income_type', 'education'])['days_employed'] \
    .transform('median'))
data.isna().sum()


children               0
days_employed          0
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [42]:
data['total_income']=data['total_income'] \
    .fillna(data.groupby(['income_type', 'education'])['total_income'] \
    .transform('median'))
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [43]:
#приведем значения в столбце 'education' к нижнему  регисту:
data['education']=data['education'].str.lower()
data['education'].value_counts()

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [44]:
data['dob_years'].value_counts()

35    617
40    609
41    607
34    603
38    598
42    597
33    581
39    573
31    560
36    555
44    547
29    545
30    540
48    538
37    537
50    514
43    513
32    510
49    508
28    503
45    497
27    493
56    487
52    484
47    480
54    479
46    475
58    461
57    460
53    459
51    448
59    444
55    443
26    408
60    377
25    357
61    355
62    352
63    269
64    265
24    264
23    254
65    194
66    183
22    183
67    167
21    111
0     101
68     99
69     85
70     65
71     58
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

In [46]:
data['age_group']=pd.qcut(data['dob_years'],[0, .25, .5, .75, 1.], labels=['до 19 лет', 'от 19-29', 'от 30-45', 'более 45' ],duplicates='raise') 
age_group.value_counts()

до 19 лет    5475
от 30-45     5463
от 19-29     5296
более 45     5291
Name: dob_years, dtype: int64

In [47]:
#добавим колонку в таблицу и применим функцию ко всему датасету:
#data['age_group'] = data['dob_years'].apply(age_group)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,от 19-29
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,от 19-29
2,0,-5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,до 19 лет
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,до 19 лет
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,от 30-45
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья,до 19 лет
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем,от 30-45
7,0,-152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование,от 30-45
8,2,-6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы,от 19-29
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи,от 19-29


In [48]:
#посчитаем количество людей по возрастной группе:
#data['age_group'].value_counts()

В категорию до 19 лет попали 101 человек, данное число совпадает с тем , что мы посчитали ранее (кол-во людей с возрастом 0). По всемй видимости это одни и те же люди, заполним данные 0 средним по группе.

In [49]:
#заполним 0 в 'dob_years' средним значением с группировкой по типу занятост:
data['dob_years']=data['dob_years'].replace(0,(data.groupby('income_type')['dob_years'].transform('mean')))
data[data['dob_years']==0].count()[0]

0

Внесем изменения в столбце `children`

Исправим число 20 на кол-во 2, а -1 на 0 детей.

In [50]:
data['children']=data['children'].replace(-1,0)
data['children']=data['children'].replace(20,2)

In [51]:
data['children'].value_counts()

0    14196
1     4818
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

**Вывод**




Общий трудовой стаж в столбце `days_employed` нам не так важен для проверки гипотез, поэтому данный столбец оставили без измнений.

Пропуски в `'total_income'` нам важны, это одна из гипотез для проверки --`Есть ли зависимость между уровнем дохода и возвратом кредита в срок?` Заполнили эти пропуски медианным значением после группировки по `'income_type', 'education'`. 

Значения столбца `education` привели  к нижнему  регисту.
Значения равные '0' столбца `dob_years` заполним данными средним по группе `income_type`.

Внесем изменения в столбце `children`.

### Замена типа данных

In [52]:
#Заменим тип данных столбца total_income в целочисленный:
data['total_income']=data['total_income'].astype(int)

In [53]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null category
dtypes: category(1), float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Столбцы `total_income` перевели в 'int' с помощью функции astype. Остальные данные отображаются корректно.
Столбец `days_employed` не трогаем, в исследовании использовать не будем.



In [54]:
#Заменим тип данных столбца days_employed в целочисленный:
data['days_employed']=data['days_employed'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 13 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
age_group           21525 non-null category
dtypes: category(1), float64(1), int64(6), object(5)
memory usage: 2.0+ MB


### Обработка дубликатов

Посчитаем явные дубликаты в таблице и удалим их из таблицы.

In [55]:
# подсчёт явных дубликатов
print("Количество дубликатов: {}".format(data.duplicated().sum()))

Количество дубликатов: 54


In [56]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых):
data=data.drop_duplicates().reset_index(drop=True)

In [57]:
print("Количество пропусков: {}".format(data.duplicated().sum()))

Количество пропусков: 0


**Вывод**

Дубликаты возможно задвоились, поэтому от явных дубликатов избавились.

### Лемматизация

In [58]:
# Посчитаем варианты целей кредита:

data['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
покупка жилья для сдачи                   652
операции с жильем                         652
операции с коммерческой недвижимостью     650
покупка жилья                             646
жилье                                     646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Из всех вариантов можно выделить 5 целей для получения кредита:'автомобиль' ,'образование','свадьба','недвижимость','жилье'

In [59]:
#напишем функцию для лемматизации и применим ее ко всему столбцу 'purpose':

def lemmatization(row):
    lemmas = m.lemmatize(row['purpose'])
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'недвижимость' in lemmas:
        return 'недвижимость'
   
    
data['lemma_purpose'] = data.apply(lemmatization, axis=1)
data.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,lemma_purpose
0,1,-8437,42.0,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,от 19-29,None
1,1,-4024,36.0,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,от 19-29,автомобиль
2,0,-5623,33.0,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,до 19 лет,None
3,3,-4124,32.0,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,до 19 лет,образование
4,0,340266,53.0,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,от 30-45,свадьба
5,0,-926,27.0,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,до 19 лет,None
6,0,-2879,43.0,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,от 30-45,None
7,0,-152,50.0,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,от 30-45,образование
8,2,-6929,35.0,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,от 19-29,свадьба
9,0,-2188,41.0,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,от 19-29,None


In [60]:
# Посчитаем возможные категории кредита:

data['lemma_purpose'].value_counts()

недвижимость    6353
автомобиль      4308
образование     4014
свадьба         2335
Name: lemma_purpose, dtype: int64

**Вывод**

Выделили леммы в значениях столбца с целями получения кредита по основным ключевым словам :
'автомобиль','образование','свадьба','недвижимость','жилье'. Посчитали количество целей, согласно выделенным категориям. На операции с недвижимостью берут больше всего кредитов -29.5%.




### Категоризация данных

Проведем категоризацию по доходам столбец `total_income` и по столбцу `children`.

Для категоризации по доходам я решила поделить людей на такие группы:
- >= 50000- низкий доход
- 50000 < total_income <= 150000- средний доход
- 150000 < total_income <= 500000 -высокий доход
- < 500000 -богатые

In [62]:
income_group=pd.qcut(data['total_income'],[0, .25, .5, .75, 1.], labels=["низкий доход", "средний доход", "высокий доход", "богатый"],duplicates='raise') 
income_group.value_counts()

средний доход    5369
богатый          5368
низкий доход     5368
высокий доход    5366
Name: total_income, dtype: int64

In [63]:
#посчитаем количетсво людей, относящихся к каждой выделенной категории:
data['income_group'].value_counts()

средний доход    11124
высокий доход     9753
низкий доход       372
богатый            222
Name: income_group, dtype: int64

In [64]:
data.sort_values('income_group')

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,age_group,lemma_purpose,income_group
10555,0,365907,67.0,высшее,0,женат / замужем,0,M,пенсионер,0,552256,покупка недвижимости,более 45,недвижимость,богатый
9990,2,-2205,34.0,высшее,0,женат / замужем,0,M,компаньон,0,690013,на покупку автомобиля,от 19-29,автомобиль,богатый
9995,0,-1979,58.0,среднее,1,женат / замужем,0,M,сотрудник,0,1097954,покупка своего жилья,более 45,None,богатый
16258,0,-6070,54.0,высшее,0,женат / замужем,0,F,компаньон,0,521420,операции со своей недвижимостью,более 45,недвижимость,богатый
19944,0,-1872,56.0,среднее,1,гражданский брак,1,F,компаньон,0,509351,на проведение свадьбы,более 45,свадьба,богатый
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12349,0,393246,57.0,среднее,1,женат / замужем,0,F,пенсионер,0,147247,покупка недвижимости,более 45,недвижимость,средний доход
4266,0,-9330,44.0,среднее,1,женат / замужем,0,F,сотрудник,0,81445,покупка жилья для сдачи,от 30-45,None,средний доход
4265,0,362034,70.0,среднее,1,женат / замужем,0,M,пенсионер,0,94266,операции с недвижимостью,более 45,недвижимость,средний доход
12355,1,-683,23.0,среднее,1,гражданский брак,1,F,сотрудник,0,127029,приобретение автомобиля,до 19 лет,автомобиль,средний доход


Категоризация по столбцу `children`

- >1- нет детей
- <=2 - 1-2 ребенка 
- <=3 - многодетные 

In [65]:
#напишем функцию для объединения данных в категории по кол-ву детейи создадим отдельный столбец с категориями детей, 
#и в его ячейках запишем значения:

def children_group(children):
    if children< 1:
        return 'нет детей'
    elif children <= 2:
        return '1-2'
    return 'многодетные'
data['children_group']=data['children'].apply(children_group)

In [66]:
#посчитаем количетсво людей, относящихся к каждой выделенной категории:
data['children_group'].value_counts()

нет детей      14154
1-2             6937
многодетные      380
Name: children_group, dtype: int64

Ранее была проедена категоризация столбца с целями получения кредита с помощью лемматизации, и категоризация столбца возраст клиента.Вот, что у нас получилось


In [67]:
#категоризация столбца с целями получения кредита
data['lemma_purpose'].value_counts()

недвижимость    6353
автомобиль      4308
образование     4014
свадьба         2335
Name: lemma_purpose, dtype: int64

In [68]:
#категоризация столбца возраст клиента
data['age_group'].value_counts()

до 19 лет    5469
от 30-45     5453
от 19-29     5285
более 45     5264
Name: age_group, dtype: int64

**Вывод**

Провели категоризацию:
- доходам
- кол-ву детей
- целям кредита
- возраст клиента

## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [69]:
pivot_table_children=data.pivot_table(index='children_group', columns='debt', values='total_income', aggfunc='count')
pivot_table_children

debt,0,1
children_group,,
1-2,6291,646
многодетные,349,31
нет детей,13090,1064


In [70]:
pivot_table_children=data.pivot_table(index='children_group', columns='debt', values='total_income',
                                      aggfunc='mean')
pivot_table_children

debt,0,1
children_group,,
1-2,170094.960738,163953.746130
многодетные,180271.418338,149884.548387
нет детей,163407.266921,159589.915414


In [71]:
pivot_table_children['ratio'] = data.groupby('children_group').debt.mean()
pivot_table_children

debt,0,1,ratio
children_group,,,
1-2,170094.960738,163953.746130,0.093124
многодетные,180271.418338,149884.548387,0.081579
нет детей,163407.266921,159589.915414,0.075173


**Вывод**

Заемщики, у которых нет детей, меньше склонны к просрочке по выплатам кредита.Это связано с тем, что они не обременены дополнительными расходами, и могут вовремя отдавать кредит

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [72]:
pivot_table_family_status=data.pivot_table(index='family_status', columns='debt', values='total_income', aggfunc='count')
pivot_table_family_status

debt,0,1
family_status,,
Не женат / не замужем,2536,274
в разводе,1110,85
вдовец / вдова,896,63
гражданский брак,3775,388
женат / замужем,11413,931


In [73]:
pivot_table_family_status['ratio'] = data.groupby('family_status').debt.mean()
pivot_table_family_status.sort_values('ratio', ascending=False)

debt,0,1,ratio
family_status,,,
Не женат / не замужем,2536,274,0.097509
гражданский брак,3775,388,0.093202
женат / замужем,11413,931,0.075421
в разводе,1110,85,0.071130
вдовец / вдова,896,63,0.065693


**Вывод**

Зависимость есть, заемщики,которые попадают под категорию "Не женат / не замужем" , склонны к просрочке по выплатам кредита.Заемщики,попадающие в категории "вдовец / вдова" либо в "разводе" ответственно выплачивают кредиты. Видимо понимают,что расчитывают только на себя и не копят долги.


- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [74]:
pivot_table_income=data.pivot_table(index='income_group', columns='debt', values='total_income', aggfunc='count')
pivot_table_income

debt,0,1
income_group,,
богатый,208,14
высокий доход,8997,756
низкий доход,349,23
средний доход,10176,948


In [75]:
pivot_table_income['ratio'] =data.groupby('income_group').debt.mean()
pivot_table_income.sort_values('ratio', ascending=False)

debt,0,1,ratio
income_group,,,
средний доход,10176,948,0.085221
высокий доход,8997,756,0.077515
богатый,208,14,0.063063
низкий доход,349,23,0.061828


**Вывод**

Кредит исправно и в срок выплачивают, заемщики с низким уровнем дохода. Граждане, попадающие по категорию "средний доход" с уровнем дохода < 50000 и >= 150000, более склонны к просрочкам.

Скорее всего это объясняется тем, что у кого маленький доход умеют грамотно распоржаться доходом, нежели те , кто зарабатывает больше.А возможно, банки более тщательно отбирают их по другим параметрам

- Как разные цели кредита влияют на его возврат в срок?

In [76]:
pivot_table_lemma_purpose=data.pivot_table(index='lemma_purpose', columns='debt', values='total_income', aggfunc='count')
pivot_table_lemma_purpose

debt,0,1
lemma_purpose,,
автомобиль,3905,403
недвижимость,5879,474
образование,3644,370
свадьба,2149,186


In [77]:
pivot_table_lemma_purpose['ratio'] = data.groupby('lemma_purpose').debt.mean()
pivot_table_lemma_purpose.sort_values('ratio', ascending=False)

debt,0,1,ratio
lemma_purpose,,,
автомобиль,3905,403,0.093547
образование,3644,370,0.092177
свадьба,2149,186,0.079657
недвижимость,5879,474,0.074610


**Вывод**

Кредит исправно и в срок выплачивают, заемщики, у которых долг по различным операциям с недвижимостью. Менее ответственно относятся с погашением кредита заемщики, берущие кредиты на покупку автомобилей.
Кредит исправно и в срок выплачивают, заемщики, у которых долг по различным операциям с недвижимостью. Менее ответственно относятся с погашением кредита заемщики, берущие кредиты на покупку автомобилей. Возможно это связано с тем, что кредиты на недвижимость это очень серьезно и суммы крупные, проценты по просрочке так же будут расти и выплачивать будет сложнее. И без жилья никто не хочет остаться.

Дополнительно проверим гипотезу влияет ли возврат кредита от возраста клиента.

In [78]:
pivot_table_age_group=data.pivot_table(index='age_group', columns='debt', values='total_income', aggfunc='count')
pivot_table_age_group

debt,0,1
age_group,,
до 19 лет,4874,595
от 19-29,4832,453
от 30-45,5054,399
более 45,4970,294


In [79]:
pivot_table_age_group['ratio'] =data.groupby('age_group').debt.mean()
pivot_table_age_group.sort_values('ratio', ascending=False)

debt,0,1,ratio
age_group,,,
до 19 лет,4874,595,0.108795
от 19-29,4832,453,0.085714
от 30-45,5054,399,0.073171
более 45,4970,294,0.055851


**Вывод**

Действительно клиенты в возрасте от 19-29 лет, более склонны к просрочке кредита, нежели клиенты в возрасте более 45 лет. Это объясняется тем,что взрослые люди, более ответственны и имеют стабильный доход. 

## Шаг 4. Общий вывод

Был проведен анализ по данным банка о платёжеспособности клиентов. Опираясь на проведенные исследования, можно сделать следующие выводы,что семейное положение и количество детей клиента влияют на факт погашения кредита в срок. 
Заемщики,которые находятся в разводе/вдовец(ва), в возрасте старше 45 лет и не имеющие детей ответственно возвращают долги банку.
